In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
%precision 3
%matplotlib inline

In [2]:
# t検定
junk_food = pd.read_csv("./data/3-8-1-junk-food-weight.csv")["weight"]
print(junk_food.head())

# ある標本抽出によって得られた標本分布から決まるt値
mu = sp.mean(junk_food)
print(mu)
df = len(junk_food) - 1
print(df)
sigma = sp.std(junk_food, ddof=1)
print(sigma)
se = sigma / sp.sqrt(len(junk_food))
print(se)

t_value = (mu - 50) / se
print(t_value)

0    58.529820
1    52.353039
2    74.446169
3    52.983263
4    55.876879
Name: weight, dtype: float64
55.38496619666667
19
8.756118777591023
1.957927680575589
2.750339683171343


In [3]:
# t分布からp値を計算

# あるt値を下回る累積確率　
alpha = stats.t.cdf(t_value, df=df)
print((1 - alpha) * 2)
# p値が0.05を下回っているので、今回の検定でスナック菓子50kgと有意差がある

0.012725590012524046


In [4]:
# stats.ttest_1samp()
stats.ttest_1samp(junk_food, 50)

Ttest_1sampResult(statistic=2.750339683171343, pvalue=0.012725590012524182)

In [5]:
# シミュレーションによるp値の計算
# p値は「帰無仮説が正しいと仮定して、何度も標本抽出〜ｔ値計算を繰り返したとき、ｔ標本と同じかそれより大きなｔ値が得られる割合」と解釈できる。

size = len(junk_food)
sigma = sp.std(junk_food, ddof=1)

# 母集団分布を平均50の正規分布と仮定して、50000回のt値を計算する
t_value_array = np.zeros(50000)

# 帰無仮説が正しいとして、標本抽出〜ｔ値計算を5万回繰り返す
np.random.seed(1)
norm_dist = stats.norm(loc=50, scale=sigma)
for i in range(0, 50000):
    sample = norm_dist.rvs(size=size) # サンプリング
    sample_mean = sp.mean(sample)
    sample_std = sp.std(sample, ddof=1)
    sample_se = sample_std / sp.sqrt(size) # 標準誤差
    t_value_array[i] = (sample_mean - 50) / sample_se


In [6]:
# 50000個のt値のうち、t標本を上回った割合
(sum(t_value_array > t_value) / 50000) * 2
# 理論値とほぼ同じ

0.01324

In [7]:
"""対応のあるt検定・・・・「同じ対象を異なった条件で2回測定して、その違いを見る」"""
# -> 差分の平均値が0と異なるかどうかをチェック

paired_test_data = pd.read_csv("./data/3-9-1-paired-t-test.csv")
print(paired_test_data)

# 帰無仮説：薬を飲む前と後で体温は変わらない
# 対立仮説：薬を飲む前と後で体温は変わる
# 有意水準5%で検定を行う

  person medicine  body_temperature
0      A   before              36.2
1      B   before              36.2
2      C   before              35.3
3      D   before              36.1
4      E   before              36.1
5      A    after              36.8
6      B    after              36.1
7      C    after              36.8
8      D    after              37.1
9      E    after              36.9


In [8]:
# 前後の標本平均
before = paired_test_data.query('medicine == "before"')["body_temperature"]
after = paired_test_data.query('medicine == "after"')["body_temperature"]

before = np.array(before)
after = np.array(after)

# 差
diff = after - before
print(diff)

[ 0.6 -0.1  1.5  1.   0.8]


In [9]:
# t検定を実行
stats.ttest_1samp(diff, 0)
# 有意差がある

Ttest_1sampResult(statistic=2.901693483620596, pvalue=0.044043109730074276)

In [10]:
"""対応のないt検定・・・「平均値の差」に注目"""
# 2群のt検定を行う
# t値の計算式が1群のt検定とは異なる

# 平均値
mean_bef = sp.mean(before)
mean_aft = sp.mean(after)

# 分散（ここが1群のt検定とは異なるところ）
var_bef = sp.var(before, ddof=1)
var_aft = sp.var(after, ddof=1)

# サンプルサイズ
m = len(before)
n = len(after)

# t値
t_value = (mean_aft - mean_bef) / sp.sqrt((var_aft/n + var_bef/m))
print(t_value)

3.1557282344421034


In [11]:
# t検定(等分散ではない)・・・Welchのt検定
stats.ttest_ind(after, before, equal_var=False)
# 有意差がある

Ttest_indResult(statistic=3.1557282344421034, pvalue=0.013484775682079892)

In [2]:
"""χ二乗検定"""

print(1 - sp.stats.chi2.cdf(x=6.667, df=1))

0.009821437357809604


In [3]:
# 分割表の検定
click_data = pd.read_csv("./data/3-10-1-click_data.csv")
print(click_data)

  color  click  freq
0  blue  click    20
1  blue    not   230
2   red  click    10
3   red    not    40


In [5]:
# 分割表に変換
cross = pd.pivot_table(
    data = click_data,
    values = "freq",
    aggfunc="sum",
    index="color",
    columns="click")

print(cross)

click  click  not
color            
blue      20  230
red       10   40


In [7]:
# χ二乗検定
sp.stats.chi2_contingency(cross, correction=False)

(6.666666666666666, 0.009823274507519247, 1, array([[ 25., 225.],
        [  5.,  45.]]))